In [1]:
from app.utilities.s3 import list_files
from app.utilities.database import get_job_status
from app.utilities.rabbitmq import list_all_queues
from app.utilities.logging import logger

In [2]:
get_job_status('validation/in-progress/1-20250905051437-test12.csv')

'file_queued'

In [3]:
list_files('validation/queued/')

[{'Key': 'validation/queued/1-20250905051437-test12.csv',
  'LastModified': datetime.datetime(2025, 9, 5, 5, 14, 48, 73000, tzinfo=tzutc()),
  'ETag': '"70ce360f86b41e4254b7de1a70b7652d"',
  'Size': 73,
  'StorageClass': 'STANDARD'},
 {'Key': 'validation/queued/1-20250907191002-test13.csv',
  'LastModified': datetime.datetime(2025, 9, 7, 19, 10, 32, 439000, tzinfo=tzutc()),
  'ETag': '"bfec12d988cfaee58ba1a8af52c73025"',
  'Size': 88,
  'StorageClass': 'STANDARD'}]

In [4]:
[(queue.get('name'), queue.get('messages', 0)) for queue in list_all_queues('mls.batch.pending_file_build')]

[('dummmy_q', 0)]

In [5]:
logger.info("Prototypes ran successfully!")

2025-09-09 17:33:00,300 - INFO - Prototypes ran successfully!
